In [1]:
import os, time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNet
%matplotlib inline

ModuleNotFoundError: No module named 'tensorflow'

Configs

In [2]:
img_needed_size = (224, 224)

In [3]:
# Get category names from the directory
categories = os.listdir("data/")
categories

['jute', 'maize', 'rice', 'sugarcane', 'wheat']

Load dataset(s)

In [10]:
crop_images_ds = keras.preprocessing.image_dataset_from_directory(
    "data/",
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    image_size=img_needed_size,
    shuffle=True
)

Found 804 files belonging to 5 classes.


Prepare model backbone&final model

In [5]:
MobileNet()

In [6]:
backbone_model = MobileNet(
    input_shape=(*img_needed_size, 3),
    include_top=False,
    weights="imagenet"
)
# Freeze base model's weights
backbone_model.trainable = False

final_model = keras.Sequential([
    backbone_model,
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation="tanh"),
    keras.layers.Dense(32, activation="tanh"),
    keras.layers.Dense(16, activation="tanh"),
    keras.layers.Dense(len(categories), activation="softmax")
])

In [7]:
final_model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics="accuracy"
)
final_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 flatten (Flatten)           (None, 50176)             0         
                                                                 
 dense (Dense)               (None, 64)                3211328   
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 5)                 85        
                                                        

Test the model on datasets, see initial results

In [8]:
final_model.evaluate(crop_images_ds)

26/26 [==============================] - 23s 776ms/step - loss: 1.7452 - accuracy: 0.2177


[1.74524986743927, 0.21766169369220734]

Train!

In [9]:
final_model.fit(crop_images_ds, batch_size=32, epochs=128, shuffle=True)

Epoch 1/128
26/26 [==============================] - 24s 829ms/step - loss: 2.0803 - accuracy: 0.2052
Epoch 2/128
26/26 [==============================] - 26s 1s/step - loss: 1.5561 - accuracy: 0.3109
Epoch 3/128
26/26 [==============================] - 25s 954ms/step - loss: 1.4622 - accuracy: 0.4229
Epoch 4/128
26/26 [==============================] - 24s 911ms/step - loss: 1.4189 - accuracy: 0.4453
Epoch 5/128
10/26 [==========>...................] - ETA: 14s - loss: 1.3767 - accuracy: 0.4344

KeyboardInterrupt: 

Test the model on datasets, see **trained** results

In [ ]:
final_model.evaluate(crop_images_ds)

In [ ]:
def test_model(dataset, max_sample=10):
    for X, y in crop_images_ds:
        for s in range(0, min(X.shape[0], max_sample)):
            X_sample = X[s:s+1]
            y_sample = y[s:s+1]

            # Predict
            y_pred = final_model.predict(X_sample)[0]
            print("Pred: %{0:.2f} {1:s}\nTarget: {2:s}".format(np.max(y_pred)*100, categories[np.argmax(y_pred)], categories[np.argmax(y_sample)]))
            plt.imshow(np.array(X_sample[0], dtype=np.int32))
            plt.show()
            time.sleep(3)
        break

Trained on the **kag2**, let's test it on **crop_images**

In [ ]:
test_model(crop_images_ds)